In [1]:
#standard libraries
import pandas as pd, numpy as np

#randomization
import random
from faker import Faker

In [2]:
#export the file(s)
students = pd.read_excel('./experimental/created_tables/students.xlsx',
                         index_col=0)

course_details = pd.read_excel('./experimental/created_tables/course_details.xlsx',
                               index_col=0)

class_details = pd.read_excel('./experimental/created_tables/class_details.xlsx',
                              index_col=0)

enrolment = pd.read_excel('./experimental/created_tables/enrolment.xlsx',
                         index_col=0)

In [3]:
#find dimensions
print(f'students: {students.shape}, \n course_details: {course_details.shape}, \n class_details: {class_details.shape}, \n enrolment: {enrolment.shape}')

students: (200, 6), 
 course_details: (5, 2), 
 class_details: (37, 3), 
 enrolment: (200, 6)


In [4]:
#display the data sets
display(students.head(2))
print()
display(course_details.head(2))
print()
display(class_details.head(2))
print()
display(enrolment.head(2))

,STUDENT_ID,FIRST_NAME,LAST_NAME,CITY,BIRTH_DATE,E-MAIL
0,S000000,James,Smith,Moscow,1974-08-25,jamessmith@yandex.com
1,S000001,Michael,Johnson,Nizhniy Novgorod,1970-02-13,michaeljohnson@yandex.com


,COURSE_ID,COURSE_TITLE
0,C0001,"CIRCULAR ECONOMY, RESPONSIBLE PRODUCTION AND C..."
1,C0002,SUSTAINABLE FOOD SYSTEMS


,COURSE_ID,CLASS_ID,CLASS_TITLE
0,C0001,CERPC101,"001 - CIRCULAR ECONOMY, RESPONSIBLE PRODUCTION..."
1,C0001,CERPC102,"002 - CIRCULAR ECONOMY, RESPONSIBLE PRODUCTION..."


,ID,COURSE_ID,STUDENT_ID,CANCELLED,ENROLMENT_DATE,COMPLETION_DATE
0,0,NaN,S000000,True,2023-12-22,2024-01-16
1,1,NaN,S000001,False,2023-08-27,2024-04-15


#### **MERGE STUDENTS + ENROLMENT**

In [5]:
#combine the students and enrolment
enrolled_students = students.merge(enrolment,
                                   how='left',
                                   on='STUDENT_ID')

enrolled_students.head(2)

,STUDENT_ID,FIRST_NAME,LAST_NAME,CITY,BIRTH_DATE,E-MAIL,ID,COURSE_ID,CANCELLED,ENROLMENT_DATE,COMPLETION_DATE
0,S000000,James,Smith,Moscow,1974-08-25,jamessmith@yandex.com,0,NaN,True,2023-12-22,2024-01-16
1,S000001,Michael,Johnson,Nizhniy Novgorod,1970-02-13,michaeljohnson@yandex.com,1,NaN,False,2023-08-27,2024-04-15


In [6]:
#check for null values
enrolled_students.isnull().sum()[enrolled_students.isnull().sum() > 0]

COURSE_ID    200
dtype: int64

In [7]:
#dimensions
enrolled_students.shape

(200, 11)

In [8]:
#concat vertically - 3x times
merged_thrice = pd.concat([enrolled_students, enrolled_students, enrolled_students],
                          axis=0)

merged_thrice.head(2)

,STUDENT_ID,FIRST_NAME,LAST_NAME,CITY,BIRTH_DATE,E-MAIL,ID,COURSE_ID,CANCELLED,ENROLMENT_DATE,COMPLETION_DATE
0,S000000,James,Smith,Moscow,1974-08-25,jamessmith@yandex.com,0,NaN,True,2023-12-22,2024-01-16
1,S000001,Michael,Johnson,Nizhniy Novgorod,1970-02-13,michaeljohnson@yandex.com,1,NaN,False,2023-08-27,2024-04-15


In [9]:
#dimensions
merged_thrice.shape

(600, 11)

#### **CREATE COURSE_ID BY RANDOMIZATION**

create a random list of courses (from 1-5) and distribute per student

In [10]:
#number of courses available
course_details['COURSE_ID'].nunique()

5

In [11]:
#number of courses
course_list = [1, 2, 3, 4, 5]

#relative weights to choose elements from the list with different probability
course_num = random.choices(course_list,
                            weights=(19, 21.5, 21.5, 14, 24),
                            k=600)

In [12]:
#insert new features
merged_thrice['COURSE_NUMBER'] = None

merged_thrice = merged_thrice.assign(COURSE_NUMBER=course_num)

In [13]:
#course distribution
merged_thrice['COURSE_NUMBER'].value_counts()

COURSE_NUMBER
5    155
3    142
2    128
1    106
4     69
Name: count, dtype: int64

In [14]:
#dimensions
merged_thrice.shape

(600, 12)

In [15]:
#write a function to return course_id
def create_course_id(data):
    return 'C000' + str(data['COURSE_NUMBER'])

#apply the function
merged_thrice['COURSE_ID'] = merged_thrice.apply(create_course_id,
                                             axis=1)

In [16]:
merged_thrice.head(3)

,STUDENT_ID,FIRST_NAME,LAST_NAME,CITY,BIRTH_DATE,E-MAIL,ID,COURSE_ID,CANCELLED,ENROLMENT_DATE,COMPLETION_DATE,COURSE_NUMBER
0,S000000,James,Smith,Moscow,1974-08-25,jamessmith@yandex.com,0,C0005,True,2023-12-22,2024-01-16,5
1,S000001,Michael,Johnson,Nizhniy Novgorod,1970-02-13,michaeljohnson@yandex.com,1,C0003,False,2023-08-27,2024-04-15,3
2,S000002,Robert,Williams,Rostov,1974-09-03,robertwilliams@yandex.com,2,C0005,False,2024-07-23,2024-08-17,5


#### **COMBINE CLASSES**

In [17]:
#add classes
enrolled_students_studies = merged_thrice.merge(class_details,
                                                how='outer',
                                                on=['COURSE_ID'])

In [18]:
#check for null values
enrolled_students_studies.isnull().sum()[enrolled_students_studies.isnull().sum() > 0]

Series([], dtype: int64)

In [19]:
#dimensions
enrolled_students_studies.shape

(4277, 14)

In [20]:
#no of students on file
enrolled_students_studies['STUDENT_ID'].nunique()

200

In [21]:
enrolled_students_studies.head(2)

,STUDENT_ID,FIRST_NAME,LAST_NAME,CITY,BIRTH_DATE,E-MAIL,ID,COURSE_ID,CANCELLED,ENROLMENT_DATE,COMPLETION_DATE,COURSE_NUMBER,CLASS_ID,CLASS_TITLE
0,S000008,Thomas,Moore,Krasnoyarsk,1975-11-26,thomasmoore@yandex.com,8,C0001,False,2023-12-14,2024-01-08,1,CERPC101,"001 - CIRCULAR ECONOMY, RESPONSIBLE PRODUCTION..."
1,S000008,Thomas,Moore,Krasnoyarsk,1975-11-26,thomasmoore@yandex.com,8,C0001,False,2023-12-14,2024-01-08,1,CERPC102,"002 - CIRCULAR ECONOMY, RESPONSIBLE PRODUCTION..."


#### **COMBINE COURSES**

In [22]:
#add courses
student_study = enrolled_students_studies.merge(course_details,
                                                how='outer',
                                                on=['COURSE_ID'])

In [23]:
#check for null values
student_study.isnull().sum()[student_study.isnull().sum() > 0]

Series([], dtype: int64)

In [24]:
#dimensions
student_study.shape

(4277, 15)

In [25]:
#no of students on file
student_study['STUDENT_ID'].nunique()

200

In [26]:
student_study.head(2)

,STUDENT_ID,FIRST_NAME,LAST_NAME,CITY,BIRTH_DATE,E-MAIL,ID,COURSE_ID,CANCELLED,ENROLMENT_DATE,COMPLETION_DATE,COURSE_NUMBER,CLASS_ID,CLASS_TITLE,COURSE_TITLE
0,S000008,Thomas,Moore,Krasnoyarsk,1975-11-26,thomasmoore@yandex.com,8,C0001,False,2023-12-14,2024-01-08,1,CERPC101,"001 - CIRCULAR ECONOMY, RESPONSIBLE PRODUCTION...","CIRCULAR ECONOMY, RESPONSIBLE PRODUCTION AND C..."
1,S000008,Thomas,Moore,Krasnoyarsk,1975-11-26,thomasmoore@yandex.com,8,C0001,False,2023-12-14,2024-01-08,1,CERPC102,"002 - CIRCULAR ECONOMY, RESPONSIBLE PRODUCTION...","CIRCULAR ECONOMY, RESPONSIBLE PRODUCTION AND C..."


In [27]:
#remove uninformative features
student_study = student_study.drop(['COURSE_NUMBER', 'CLASS_TITLE', 'COURSE_TITLE'],
                                   axis=1,
                                   errors='ignore')

#### **CREATE COURSE_START_DATE / COURSE_END_DATE**

In [47]:
#slice the table
slice_course_dates = student_study.iloc[:, [0, 6, 7, 9, 10]]
slice_course_dates.head(2)

,STUDENT_ID,ID,COURSE_ID,ENROLMENT_DATE,COMPLETION_DATE
0,S000008,8,C0001,2023-12-14,2024-01-08
1,S000008,8,C0001,2023-12-14,2024-01-08


In [48]:
#check for duplicate values
dupl_columns = list(slice_course_dates.columns)

mask = slice_course_dates.duplicated(subset=dupl_columns)
slice_duplicates = slice_course_dates[mask]
print(f'Number of Duplicates: {slice_duplicates.shape[0]}')

Number of Duplicates: 3800


In [49]:
#remove duplicates
slice_course_dates = slice_course_dates.drop_duplicates(subset=dupl_columns)
print(f'New Dimensions: {slice_course_dates.shape[0]}')

New Dimensions: 477


In [50]:
#reset the index
slice_course_dates = slice_course_dates.reset_index()

#remove uninformative feature
slice_course_dates = slice_course_dates.drop('index',
                                             axis=1,
                                             errors='ignore')

In [51]:
slice_course_dates.head(2)

,STUDENT_ID,ID,COURSE_ID,ENROLMENT_DATE,COMPLETION_DATE
0,S000008,8,C0001,2023-12-14,2024-01-08
1,S000009,9,C0001,2023-05-20,2024-04-19


In [52]:
from datetime import date

#write a function to create the course start date
def create_course_start(data):
    #create the object class
    fake_start = Faker()
    
    #randomize between enrolment and completion date
    return fake_start.date_between(start_date=data['ENROLMENT_DATE'],
                                   end_date=data['COMPLETION_DATE'])

In [53]:
#apply the function
slice_course_dates['COURSE_START_DATE'] = slice_course_dates.apply(create_course_start,
                                                                   axis=1)

In [54]:
from datetime import date

#write a function to create the course end date
def create_course_end(data):
    #create the object class
    fake_end = Faker()
    
    #randomize between course start and completion date
    return fake_end.date_between(start_date=data['COURSE_START_DATE'],
                                 end_date=data['COMPLETION_DATE'])

In [55]:
#apply the function
slice_course_dates['COURSE_END_DATE'] = slice_course_dates.apply(create_course_end,
                                                                 axis=1)

LOGIC CHECK

In [56]:
#enrolment cannot be after completion
slice_course_dates[slice_course_dates['ENROLMENT_DATE'] > slice_course_dates['COMPLETION_DATE']]

,STUDENT_ID,ID,COURSE_ID,ENROLMENT_DATE,COMPLETION_DATE,COURSE_START_DATE,COURSE_END_DATE


In [57]:
#start date cannot be before enrolment
slice_course_dates[slice_course_dates['COURSE_START_DATE'] < slice_course_dates['ENROLMENT_DATE']]

,STUDENT_ID,ID,COURSE_ID,ENROLMENT_DATE,COMPLETION_DATE,COURSE_START_DATE,COURSE_END_DATE


In [58]:
#start cannot be after completion
slice_course_dates[slice_course_dates['COURSE_START_DATE'] > slice_course_dates['COURSE_END_DATE']]

,STUDENT_ID,ID,COURSE_ID,ENROLMENT_DATE,COMPLETION_DATE,COURSE_START_DATE,COURSE_END_DATE


In [59]:
student_study.shape

(4277, 12)

In [61]:
slice_course_dates.head(2)

,STUDENT_ID,ID,COURSE_ID,ENROLMENT_DATE,COMPLETION_DATE,COURSE_START_DATE,COURSE_END_DATE
0,S000008,8,C0001,2023-12-14,2024-01-08,2023-12-21,2023-12-25
1,S000009,9,C0001,2023-05-20,2024-04-19,2023-08-18,2023-12-08


In [62]:
#add course_dates to the main merged table
student_course_data = student_study.merge(slice_course_dates,
                                          how='inner',
                                          on=['STUDENT_ID', 'COURSE_ID', 'ID', 'ENROLMENT_DATE', 'COMPLETION_DATE'])

In [63]:
student_course_data.head(2)

,STUDENT_ID,FIRST_NAME,LAST_NAME,CITY,BIRTH_DATE,E-MAIL,ID,COURSE_ID,CANCELLED,ENROLMENT_DATE,COMPLETION_DATE,CLASS_ID,COURSE_START_DATE,COURSE_END_DATE
0,S000008,Thomas,Moore,Krasnoyarsk,1975-11-26,thomasmoore@yandex.com,8,C0001,False,2023-12-14,2024-01-08,CERPC101,2023-12-21,2023-12-25
1,S000008,Thomas,Moore,Krasnoyarsk,1975-11-26,thomasmoore@yandex.com,8,C0001,False,2023-12-14,2024-01-08,CERPC102,2023-12-21,2023-12-25


In [64]:
student_course_data.head(2)

,STUDENT_ID,FIRST_NAME,LAST_NAME,CITY,BIRTH_DATE,E-MAIL,ID,COURSE_ID,CANCELLED,ENROLMENT_DATE,COMPLETION_DATE,CLASS_ID,COURSE_START_DATE,COURSE_END_DATE
0,S000008,Thomas,Moore,Krasnoyarsk,1975-11-26,thomasmoore@yandex.com,8,C0001,False,2023-12-14,2024-01-08,CERPC101,2023-12-21,2023-12-25
1,S000008,Thomas,Moore,Krasnoyarsk,1975-11-26,thomasmoore@yandex.com,8,C0001,False,2023-12-14,2024-01-08,CERPC102,2023-12-21,2023-12-25


In [65]:
student_course_data.shape

(4277, 14)

#### **CREATE CLASS_START_DATE / CLASS_END_DATE**

In [67]:
#slice the table
slice_class_dates = student_course_data.iloc[:, [0, 6, 7, 9, 10, 11, 12, 13]]
slice_class_dates.head(2)

,STUDENT_ID,ID,COURSE_ID,ENROLMENT_DATE,COMPLETION_DATE,CLASS_ID,COURSE_START_DATE,COURSE_END_DATE
0,S000008,8,C0001,2023-12-14,2024-01-08,CERPC101,2023-12-21,2023-12-25
1,S000008,8,C0001,2023-12-14,2024-01-08,CERPC102,2023-12-21,2023-12-25


In [68]:
#check for duplicate values
dupl_columns = list(slice_class_dates.columns)

mask = slice_class_dates.duplicated(subset=dupl_columns)
slice_class_duplicates = slice_class_dates[mask]
print(f'Number of Duplicates: {slice_class_duplicates.shape[0]}')

Number of Duplicates: 860


In [69]:
#remove duplicates
slice_class_dates = slice_class_dates.drop_duplicates(subset=dupl_columns)
print(f'New Dimensions: {slice_class_dates.shape[0]}')

New Dimensions: 3417


In [70]:
#reset the index
slice_class_dates = slice_class_dates.reset_index()

#remove uninformative feature
slice_class_dates = slice_class_dates.drop('index',
                                           axis=1,
                                           errors='ignore')

In [71]:
slice_class_dates.head(2)

,STUDENT_ID,ID,COURSE_ID,ENROLMENT_DATE,COMPLETION_DATE,CLASS_ID,COURSE_START_DATE,COURSE_END_DATE
0,S000008,8,C0001,2023-12-14,2024-01-08,CERPC101,2023-12-21,2023-12-25
1,S000008,8,C0001,2023-12-14,2024-01-08,CERPC102,2023-12-21,2023-12-25


In [72]:
from datetime import date

#write a function to create the class start date
def create_class_start(data):
    #create the object class
    fake_start = Faker()
    
    #randomize between course start and course end
    return fake_start.date_between(start_date=data['COURSE_START_DATE'],
                                   end_date=data['COURSE_END_DATE'])

In [73]:
#apply the function
slice_class_dates['CLASS_START_DATE'] = slice_class_dates.apply(create_class_start,
                                                                axis=1)

In [74]:
from datetime import date

#write a function to create the class end date
def create_class_end(data):
    #create the object class
    fake_end = Faker()
    
    #randomize between class start and course end
    return fake_end.date_between(start_date=data['CLASS_START_DATE'],
                                 end_date=data['COURSE_END_DATE'])

In [75]:
#apply the function
slice_class_dates['CLASS_END_DATE'] = slice_class_dates.apply(create_class_end,
                                                              axis=1)

LOGIC CHECK

In [76]:
#class start cannot be after course end
slice_class_dates[slice_class_dates['CLASS_START_DATE'] > slice_class_dates['COURSE_END_DATE']]

,STUDENT_ID,ID,COURSE_ID,ENROLMENT_DATE,COMPLETION_DATE,CLASS_ID,COURSE_START_DATE,COURSE_END_DATE,CLASS_START_DATE,CLASS_END_DATE


In [77]:
#class end cannot be before class start
slice_class_dates[slice_class_dates['CLASS_START_DATE'] > slice_class_dates['CLASS_END_DATE']]

,STUDENT_ID,ID,COURSE_ID,ENROLMENT_DATE,COMPLETION_DATE,CLASS_ID,COURSE_START_DATE,COURSE_END_DATE,CLASS_START_DATE,CLASS_END_DATE


In [78]:
student_course_data.shape

(4277, 14)

In [79]:
#add class dates to the main merged table
student_class_data = student_course_data.merge(slice_class_dates,
                                               how='inner',
                                               on=['STUDENT_ID', 'ID',
                                                   'ENROLMENT_DATE', 'COMPLETION_DATE',
                                                   'CLASS_ID',
                                                   'COURSE_ID', 'COURSE_START_DATE', 'COURSE_END_DATE'])

In [80]:
student_class_data.head(2)

,STUDENT_ID,FIRST_NAME,LAST_NAME,CITY,BIRTH_DATE,E-MAIL,ID,COURSE_ID,CANCELLED,ENROLMENT_DATE,COMPLETION_DATE,CLASS_ID,COURSE_START_DATE,COURSE_END_DATE,CLASS_START_DATE,CLASS_END_DATE
0,S000008,Thomas,Moore,Krasnoyarsk,1975-11-26,thomasmoore@yandex.com,8,C0001,False,2023-12-14,2024-01-08,CERPC101,2023-12-21,2023-12-25,2023-12-22,2023-12-23
1,S000008,Thomas,Moore,Krasnoyarsk,1975-11-26,thomasmoore@yandex.com,8,C0001,False,2023-12-14,2024-01-08,CERPC102,2023-12-21,2023-12-25,2023-12-21,2023-12-24


In [81]:
student_class_data.shape

(4277, 16)

#### **CREATE TEST_NUMBER**

create test_number by combining course_id + T000 + class_id

In [82]:
#slice two characters from class id
student_class_data['TNUM'] = student_class_data['CLASS_ID'].apply(lambda x: str(x)[-2:])

In [84]:
#write a function to create the test id
def create_test_number(data):
    return str(data['COURSE_ID']) + 'T000' + str(data['TNUM'])

#apply a function
student_class_data['TEST_NUMBER'] = student_class_data.apply(create_test_number,
                                                             axis=1)

In [85]:
student_class_data.head(2)

,STUDENT_ID,FIRST_NAME,LAST_NAME,CITY,BIRTH_DATE,E-MAIL,ID,COURSE_ID,CANCELLED,ENROLMENT_DATE,COMPLETION_DATE,CLASS_ID,COURSE_START_DATE,COURSE_END_DATE,CLASS_START_DATE,CLASS_END_DATE,TNUM,TEST_NUMBER
0,S000008,Thomas,Moore,Krasnoyarsk,1975-11-26,thomasmoore@yandex.com,8,C0001,False,2023-12-14,2024-01-08,CERPC101,2023-12-21,2023-12-25,2023-12-22,2023-12-23,01,C0001T00001
1,S000008,Thomas,Moore,Krasnoyarsk,1975-11-26,thomasmoore@yandex.com,8,C0001,False,2023-12-14,2024-01-08,CERPC102,2023-12-21,2023-12-25,2023-12-21,2023-12-24,02,C0001T00002


CREATE TESTS TABLE

In [88]:
#combine courses + classes
tests = course_details.merge(class_details,
                             how='left',
                             on=['COURSE_ID'])

In [91]:
#slice two characters from class id
tests['TNUM'] = tests['CLASS_ID'].apply(lambda x: str(x)[-2:])

In [93]:
#apply a function
tests['TEST_NUMBER'] = tests.apply(create_test_number,
                                   axis=1)

In [95]:
#remove uninformative feature(s)
tests = tests.drop('TNUM',
                   axis=1,
                   errors='ignore')

In [96]:
tests.head(2)

,COURSE_ID,COURSE_TITLE,CLASS_ID,CLASS_TITLE,TEST_NUMBER
0,C0001,"CIRCULAR ECONOMY, RESPONSIBLE PRODUCTION AND C...",CERPC101,"001 - CIRCULAR ECONOMY, RESPONSIBLE PRODUCTION...",C0001T00001
1,C0001,"CIRCULAR ECONOMY, RESPONSIBLE PRODUCTION AND C...",CERPC102,"002 - CIRCULAR ECONOMY, RESPONSIBLE PRODUCTION...",C0001T00002


EXPORT TABLE

In [97]:
#create a file name
xlx_name = 'test_details.xlsx'

#export to excel
tests.to_excel('./experimental/created_tables/' + xlx_name,
               sheet_name='test_details')

#### **CREATE TEST_SCORE**

create a list of test scores with a different set of probabilities

In [100]:
#75-100%, 0-100%
test_score1, test_score2 = [75, 80, 85, 90, 95, 100], [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

#probability
prob1, prob2 = 100 / len(test_score1), 100 / len(test_score2)

#split
split1 = round(student_class_data.shape[0] / 2, 0)
split2 = student_class_data.shape[0] - split1

#relative weights to choose elements from the list with different probability
scores1 = random.choices(test_score1,
                         weights=(prob1, prob1, prob1, prob1, prob1, prob1),
                         k=int(split1))

#relative weights to choose elements from the list with different probability
scores2 = random.choices(test_score2,
                         weights=(prob2, prob2, prob2, prob2, prob2, prob2, prob2, prob2, prob2, prob2,
                                  prob2, prob2, prob2, prob2, prob2, prob2, prob2, prob2, prob2, prob2,
                                  prob2),
                         k=int(split2))

#add two lists together
combine_test_scores = scores1 + scores2

In [101]:
#insert a new feature
student_class_data['TEST_SCORE'] = None

student_class_data = student_class_data.assign(TEST_SCORE=combine_test_scores)

In [102]:
#write a function for test_status
def pass_or_fail(data):
    if data['TEST_SCORE'] > 50:
        return 'PASS'
    else:
        return 'FAIL'

#create a new feature
student_class_data['TEST_STATUS'] = student_class_data.apply(pass_or_fail,
                                                             axis=1)

#### **CREATE COURSE_STATUS**

In [103]:
#group the test_scores by course (per student)
average_score_per_course = student_class_data.groupby(['STUDENT_ID', 'COURSE_ID'])['TEST_SCORE'].mean().round(0)
average_score_per_course[:5]

STUDENT_ID  COURSE_ID
S000000     C0001        86.0
            C0004        48.0
            C0005        51.0
S000001     C0003        61.0
            C0005        35.0
Name: TEST_SCORE, dtype: float64

In [104]:
#convert series to dataframe
average_score_per_course = average_score_per_course.to_frame()
type(average_score_per_course)

pandas.core.frame.DataFrame

In [105]:
#write a function for course_status
def course_pass(data):
    if data['TEST_SCORE'] >= 50:
        return 'PASS'
    else:
        return 'FAIL'
    
#apply the function
average_score_per_course['COURSE_STATUS'] = average_score_per_course.apply(course_pass,
                                                                           axis=1)

In [106]:
average_score_per_course.shape

(477, 2)

In [107]:
#create an index to fill in the blanks
average_score_per_course = average_score_per_course.reset_index()

In [108]:
#confirm that the change was applied correctly
average_score_per_course.head(3)

,STUDENT_ID,COURSE_ID,TEST_SCORE,COURSE_STATUS
0,S000000,C0001,86.0,PASS
1,S000000,C0004,48.0,FAIL
2,S000000,C0005,51.0,PASS


In [109]:
#rename the column
average_score_per_course = average_score_per_course.rename(columns={'TEST_SCORE': 'COURSE_SCORE',
                                                                    })

In [110]:
average_score_per_course.shape

(477, 4)

In [111]:
#add course_score and course_status to the main merged table
student_data = student_class_data.merge(average_score_per_course,
                                        how='outer',
                                        on=['STUDENT_ID', 'COURSE_ID'])

In [112]:
student_data.shape

(4277, 22)

#### **CREATE ACTIVE STUDENTS**

In [113]:
#create a new feature: student status
student_data['STUDENT_STATUS'] = student_data['CANCELLED'].apply(lambda x: 'ACTIVE' if x == False else 'INACTIVE')

In [116]:
#remove uninformative feature(s)
student_data = student_data.drop(['TNUM'],
                                 axis=1,
                                 errors='ignore')

In [117]:
student_data.head(2)

,STUDENT_ID,FIRST_NAME,LAST_NAME,CITY,BIRTH_DATE,E-MAIL,ID,COURSE_ID,CANCELLED,ENROLMENT_DATE,...,COURSE_START_DATE,COURSE_END_DATE,CLASS_START_DATE,CLASS_END_DATE,TEST_NUMBER,TEST_SCORE,TEST_STATUS,COURSE_SCORE,COURSE_STATUS,STUDENT_STATUS
0,S000000,James,Smith,Moscow,1974-08-25,jamessmith@yandex.com,0,C0001,True,2023-12-22,...,2023-12-28,2024-01-01,2023-12-30,2023-12-31,C0001T00001,100,PASS,86.0,PASS,INACTIVE
1,S000000,James,Smith,Moscow,1974-08-25,jamessmith@yandex.com,0,C0001,True,2023-12-22,...,2023-12-28,2024-01-01,2023-12-29,2023-12-29,C0001T00002,75,PASS,86.0,PASS,INACTIVE


#### **CREATE ACHIEVEMENTS AND POINTS**

SCORING: 0-100

ACHIEVEMENTS:
> GOLD: 90-100% \
> SILVER: 80-89% \
> BRONZE: 70-79%

POINTS:
- GOLD (100)
- SILVER (75)
- BRONZE (50)

In [118]:
#write a function for achievements
def create_achievements(data):
    if 90 <= data['TEST_SCORE'] <= 100:
        return 'GOLD'
    elif 80 <= data['TEST_SCORE'] <= 89:
        return 'SILVER'
    elif 70 <= data['TEST_SCORE'] <= 79:
        return 'BRONZE'
    else:
        return None

#apply the function
student_data['ACHIEVEMENT'] = student_data.apply(create_achievements,
                                                 axis=1)

In [119]:
#write a function for points
def get_points(data):
    if data['ACHIEVEMENT'] == 'GOLD':
        return 100
    elif data['ACHIEVEMENT'] == 'SILVER':
        return 75
    elif data['ACHIEVEMENT'] == 'BRONZE':
        return 50
    else:
        return 0

#apply the function
student_data['POINTS'] = student_data.apply(get_points,
                                            axis=1)

In [120]:
student_data.head(2)

,STUDENT_ID,FIRST_NAME,LAST_NAME,CITY,BIRTH_DATE,E-MAIL,ID,COURSE_ID,CANCELLED,ENROLMENT_DATE,...,CLASS_START_DATE,CLASS_END_DATE,TEST_NUMBER,TEST_SCORE,TEST_STATUS,COURSE_SCORE,COURSE_STATUS,STUDENT_STATUS,ACHIEVEMENT,POINTS
0,S000000,James,Smith,Moscow,1974-08-25,jamessmith@yandex.com,0,C0001,True,2023-12-22,...,2023-12-30,2023-12-31,C0001T00001,100,PASS,86.0,PASS,INACTIVE,GOLD,100
1,S000000,James,Smith,Moscow,1974-08-25,jamessmith@yandex.com,0,C0001,True,2023-12-22,...,2023-12-29,2023-12-29,C0001T00002,75,PASS,86.0,PASS,INACTIVE,BRONZE,50


In [121]:
#check for null values
student_data.isnull().sum()[student_data.isnull().sum() > 0]

ACHIEVEMENT    1398
dtype: int64

#### **CREATE UNFINISHED COURSES**

In [122]:
student_data = student_data.reset_index()

remove some records to create unfinished courses

In [123]:
#slice two characters from class id
student_data['TNUM'] = student_data['CLASS_ID'].apply(lambda x: str(x)[-2:])

COURSE ONE

In [124]:
#filter by course one and last two classes
course_one_indx = student_data[(student_data['COURSE_ID'] == 'C0001') &
                               ((student_data['TNUM'] == '09') | (student_data['TNUM'] == '10'))]
course_one_indx.index

Index([   8,    9,  132,  133,  152,  153,  174,  175,  196,  197,
       ...
       4098, 4099, 4159, 4160, 4181, 4182, 4222, 4223, 4232, 4233],
      dtype='int64', length=212)

In [125]:
#create a list to store course one records to remove
delete_one = []

#find records divisable by 4 without a remainder
for r in course_one_indx.index.to_list():
    if r % 4 == 0:
        delete_one.append(r)

delete_one
len(delete_one)

49

COURSE TWO AND THREE

In [126]:
#filter by course two and three and last two classes
courses_two_three_indx = student_data[((student_data['COURSE_ID'] == 'C0002') | (student_data['COURSE_ID'] == 'C0003')) &
                                      ((student_data['TNUM'] == '06') | (student_data['TNUM'] == '07'))]
courses_two_three_indx.index

Index([  28,   29,   35,   36,   47,   48,   54,   55,  108,  109,
       ...
       4202, 4203, 4244, 4245, 4251, 4252, 4258, 4259, 4265, 4266],
      dtype='int64', length=540)

In [127]:
#create a list to store course two and three records to remove
delete_two_three = []

#find records divisable by 3 without a remainder
for r in courses_two_three_indx.index.to_list():
    if r % 3 == 0:
        delete_two_three.append(r)

delete_two_three
len(delete_two_three)

185

COURSE FOUR

In [128]:
#filter by course four and last class
course_four_indx = student_data[(student_data['COURSE_ID'] == 'C0004') & (student_data['TNUM'] == '08')]
course_four_indx.index

Index([  17,   68,   76,   89,   97,  260,  318,  488,  816,  824,  832,  854,
        876,  894,  957,  965,  973,  996, 1004, 1029, 1044, 1150, 1273, 1288,
       1391, 1572, 1599, 1621, 1709, 1734, 1759, 1767, 1790, 1798, 2148, 2156,
       2190, 2301, 2369, 2391, 2424, 2495, 2503, 2538, 2620, 2628, 2935, 3046,
       3086, 3137, 3191, 3287, 3329, 3349, 3394, 3472, 3538, 3579, 3733, 3741,
       3763, 3800, 3903, 3942, 3996, 4040, 4048, 4070, 4133],
      dtype='int64')

In [129]:
#create a list to store course four records to remove
delete_four = []

#find records divisable by 5 without a remainder
for r in course_four_indx.index.to_list():
    if r % 5 == 0:
        delete_four.append(r)

delete_four
len(delete_four)

11

COURSE FIVE

In [130]:
#filter by course five and last class
course_five_indx = student_data[(student_data['COURSE_ID'] == 'C0005') & (student_data['TNUM'] == '05')]
course_five_indx.index

Index([  22,   41,   60,   81,  102,  138,  143,  165,  187,  300,
       ...
       4089, 4111, 4145, 4150, 4172, 4208, 4213, 4238, 4271, 4276],
      dtype='int64', length=155)

In [131]:
#create a list to store course five records to remove
delete_five = []

#find records divisable by 12 without a remainder
for r in course_five_indx.index.to_list():
    if r % 12 == 0:
        delete_five.append(r)

delete_five
len(delete_five)

6

REMOVE RECORDS TO CREATE UNFINISHED COURSES

In [132]:
student_data.shape

(4277, 26)

In [133]:
#combine into one index
delete_records = delete_one + delete_two_three + delete_four + delete_five

#remove course records
student_data = student_data.drop(student_data.index[delete_records],
                                 axis=0,
                                 errors='ignore')

In [134]:
student_data.shape

(4026, 26)

In [136]:
#remove uninformative feature(s)
student_data = student_data.drop(['index', 'TNUM'],
                                 axis=1,
                                 errors='ignore')

#### **CREATE LOGS**

In [137]:
#slice the table
attend = student_data[['STUDENT_ID', 'ID', 'COURSE_ID', 'CLASS_ID', 'CLASS_START_DATE', 'CLASS_END_DATE']]
attend.head(3)

,STUDENT_ID,ID,COURSE_ID,CLASS_ID,CLASS_START_DATE,CLASS_END_DATE
0,S000000,0,C0001,CERPC101,2023-12-30,2023-12-31
1,S000000,0,C0001,CERPC102,2023-12-29,2023-12-29
2,S000000,0,C0001,CERPC103,2023-12-30,2023-12-30


In [138]:
#check for duplicate values
dupl_columns = list(attend.columns)

mask = attend.duplicated(subset=dupl_columns)
attend_duplicates = attend[mask]
print(f'Number of Duplicates: {attend_duplicates.shape[0]}')

Number of Duplicates: 755


In [139]:
#remove duplicates
attend = attend.drop_duplicates(subset=dupl_columns)
print(f'New Dimensions: {attend.shape[0]}')

New Dimensions: 3271


In [141]:
#convert to specific format to avoid log_out < log_in
attend['CLASS_START_DATE'] = pd.to_datetime(attend['CLASS_START_DATE'],
                                       format='%Y-%m-%d %H:%M:%S')

attend['CLASS_END_DATE'] = pd.to_datetime(attend['CLASS_END_DATE'],
                                     format='%Y-%m-%d %H:%M:%S')

In [142]:
import datetime
from datetime import date

#write a function to create the log in
def create_session_start(data):
    #create the object class
    fake_timezone = Faker()
    
    #randomize between class start and class end
    return fake_timezone.date_time_between(start_date=data['CLASS_START_DATE'],
                                             end_date=data['CLASS_END_DATE'])

#apply the function
attend['LOG_IN'] = attend.apply(create_session_start,
                                axis=1)

In [143]:
#write a function to create the log out
def create_session_end(data):
    #create the object class
    fake_timezone = Faker()
    
    #randomize between log in and class end
    return fake_timezone.date_time_between(start_date=data['LOG_IN'],
                                              end_date=data['CLASS_END_DATE'])

#apply the function
attend['LOG_OUT'] = attend.apply(create_session_end,
                                 axis=1)

In [144]:
attend.head(2)

,STUDENT_ID,ID,COURSE_ID,CLASS_ID,CLASS_START_DATE,CLASS_END_DATE,LOG_IN,LOG_OUT
0,S000000,0,C0001,CERPC101,2023-12-30,2023-12-31,2023-12-30 05:20:06.953221,2023-12-30 09:16:28.753009
1,S000000,0,C0001,CERPC102,2023-12-29,2023-12-29,2023-12-29 00:00:00.445911,2023-12-29 00:00:00.620733


In [145]:
#find the session time
attend['DIF'] = abs(attend['LOG_OUT'] - attend['LOG_IN'])

#convert to string
attend['DIF'] = attend['DIF'].astype('string')

In [146]:
from datetime import datetime
from operator import attrgetter

#write a function to return session time (in minutes) to 2dp to signify seconds, i.e. 0.05 = 3 seconds (0.05 * 60 = 3)
def str_datetime_to_minute(data):
    #split the string
    days, time = data['DIF'].split(' days ')
    
    #separate elements
    hours, minutes, seconds, microseconds = attrgetter(
        'hour', 'minute', 'second', 'microsecond'
    )(datetime.strptime(time, '%H:%M:%S.%f'))
    
    #convert to minutes rounding down to 2dp
    return round(int(days) * 24 * 60
            + hours * 60
            + minutes
            + seconds / 60
            + microseconds / 60 / 1000000, 2)

In [147]:
#apply the function to create the new feature
attend['SESSION_TIME'] = attend.apply(str_datetime_to_minute,
                                      axis=1)

In [149]:
attend.head(2)

,STUDENT_ID,ID,COURSE_ID,CLASS_ID,CLASS_START_DATE,CLASS_END_DATE,LOG_IN,LOG_OUT,DIF,SESSION_TIME
0,S000000,0,C0001,CERPC101,2023-12-30,2023-12-31,2023-12-30 05:20:06.953221,2023-12-30 09:16:28.753009,0 days 03:56:21.799788,236.36
1,S000000,0,C0001,CERPC102,2023-12-29,2023-12-29,2023-12-29 00:00:00.445911,2023-12-29 00:00:00.620733,0 days 00:00:00.174822,0.00


In [148]:
#distribution
attend['SESSION_TIME'].value_counts().sort_values(ascending=True)

SESSION_TIME
54.11        1
3392.62      1
438.31       1
101.91       1
5278.96      1
          ... 
78.21        2
818.83       2
0.02        13
0.00       655
0.01       707
Name: count, Length: 1891, dtype: int64

not creating multiple sessions, so outliers possible

In [150]:
#create the sessions
sessions = list(range(1, (attend.shape[0]+1)))
len(sessions)

3271

In [151]:
#insert the new feature
attend.insert(0, 'SESSION_ID', sessions, True)

In [152]:
attend = attend.reset_index()

In [153]:
#remove uninformative feature(s)
attend = attend.drop('index',
                     axis=1,
                     errors='ignore')

In [154]:
attend.head(2)

,SESSION_ID,STUDENT_ID,ID,COURSE_ID,CLASS_ID,CLASS_START_DATE,CLASS_END_DATE,LOG_IN,LOG_OUT,DIF,SESSION_TIME
0,1,S000000,0,C0001,CERPC101,2023-12-30,2023-12-31,2023-12-30 05:20:06.953221,2023-12-30 09:16:28.753009,0 days 03:56:21.799788,236.36
1,2,S000000,0,C0001,CERPC102,2023-12-29,2023-12-29,2023-12-29 00:00:00.445911,2023-12-29 00:00:00.620733,0 days 00:00:00.174822,0.00


EXPORT TABLE

In [155]:
#create a file name
xlx_name = 'attendance.xlsx'

#export to excel
attend.to_excel('./experimental/created_tables/' + xlx_name,
                sheet_name='logging_data')

##### **EXPORT TABLES**

In [156]:
student_data.head(3)

,STUDENT_ID,FIRST_NAME,LAST_NAME,CITY,BIRTH_DATE,E-MAIL,ID,COURSE_ID,CANCELLED,ENROLMENT_DATE,...,CLASS_START_DATE,CLASS_END_DATE,TEST_NUMBER,TEST_SCORE,TEST_STATUS,COURSE_SCORE,COURSE_STATUS,STUDENT_STATUS,ACHIEVEMENT,POINTS
0,S000000,James,Smith,Moscow,1974-08-25,jamessmith@yandex.com,0,C0001,True,2023-12-22,...,2023-12-30,2023-12-31,C0001T00001,100,PASS,86.0,PASS,INACTIVE,GOLD,100
1,S000000,James,Smith,Moscow,1974-08-25,jamessmith@yandex.com,0,C0001,True,2023-12-22,...,2023-12-29,2023-12-29,C0001T00002,75,PASS,86.0,PASS,INACTIVE,BRONZE,50
2,S000000,James,Smith,Moscow,1974-08-25,jamessmith@yandex.com,0,C0001,True,2023-12-22,...,2023-12-30,2023-12-30,C0001T00003,75,PASS,86.0,PASS,INACTIVE,BRONZE,50


In [165]:
#create a file name
xlx_name = 'merged_tables.xlsx'

#export to excel
student_data.to_excel('./experimental/database_tables/' + xlx_name,
                       sheet_name='merged_tables')

TEST_SCORES

In [159]:
#slice the merged table
test_scores = student_data[['STUDENT_ID', 
                            'COURSE_ID', 'CLASS_ID',
                            'TEST_NUMBER', 'TEST_SCORE', 'TEST_STATUS',
                            'STUDENT_STATUS',
                            'ACHIEVEMENT', 'POINTS']]

test_scores.head(2)

,STUDENT_ID,COURSE_ID,CLASS_ID,TEST_NUMBER,TEST_SCORE,TEST_STATUS,STUDENT_STATUS,ACHIEVEMENT,POINTS
0,S000000,C0001,CERPC101,C0001T00001,100,PASS,INACTIVE,GOLD,100
1,S000000,C0001,CERPC102,C0001T00002,75,PASS,INACTIVE,BRONZE,50


In [166]:
#create a file name
xlx_name = 'test_scores.xlsx'

#export to excel
test_scores.to_excel('./experimental/created_tables/' + xlx_name,
                       sheet_name='test_scores')

CURRENT COURSES

In [162]:
#slice the merged table
courses = student_data[['STUDENT_ID',
                        'COURSE_ID',
                        'COURSE_START_DATE', 'COURSE_END_DATE', 'COURSE_SCORE', 'COURSE_STATUS',
                        'STUDENT_STATUS']]

courses.head(2)

,STUDENT_ID,COURSE_ID,COURSE_START_DATE,COURSE_END_DATE,COURSE_SCORE,COURSE_STATUS,STUDENT_STATUS
0,S000000,C0001,2023-12-28,2024-01-01,86.0,PASS,INACTIVE
1,S000000,C0001,2023-12-28,2024-01-01,86.0,PASS,INACTIVE


In [167]:
#create a file name
xlx_name = 'courses.xlsx'

#export to excel
courses.to_excel('./experimental/created_tables/' + xlx_name,
                 sheet_name='courses')

CURRENT CLASSES

In [163]:
#slice the merged table
classes = student_data[['STUDENT_ID',
                        'COURSE_ID', 'CLASS_ID',
                        'COURSE_START_DATE', 'COURSE_END_DATE',
                        'CLASS_START_DATE', 'CLASS_END_DATE', 'TEST_STATUS',
                        'STUDENT_STATUS']]

classes.head(2)

,STUDENT_ID,COURSE_ID,CLASS_ID,COURSE_START_DATE,COURSE_END_DATE,CLASS_START_DATE,CLASS_END_DATE,TEST_STATUS,STUDENT_STATUS
0,S000000,C0001,CERPC101,2023-12-28,2024-01-01,2023-12-30,2023-12-31,PASS,INACTIVE
1,S000000,C0001,CERPC102,2023-12-28,2024-01-01,2023-12-29,2023-12-29,PASS,INACTIVE


In [164]:
#rename the feature(s)
classes = classes.rename(columns={'TEST_STATUS': 'CLASS_STATUS',
                                  })

In [168]:
#create a file name
xlx_name = 'classes.xlsx'

#export to excel
classes.to_excel('./experimental/created_tables/' + xlx_name,
                 sheet_name='classes')